In [72]:
import os
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm

background_colour = "#000000"
img_size = 512
circumscribed_size = int(img_size / 2 * np.sqrt(2)) * 2
dir_ = "../images/stripes/test"
tag = "v2"
img_sets = 50
angles = [0, 45, 90, 135]
max_attemps = 10000

In [73]:
class CreateStripes:
    min_thickness = 10
    max_thickness = 30
    min_spacing = 5
    min_stripe_num = 2
    size = 512
    dir_path = dir_

    def __init__(self, max_stripe_num, angles):
        self.max_stripe_num = max_stripe_num
        self.angles = angles
        self.c_size = int(self.size / 2 * np.sqrt(2)) * 2

    def create_images(self):
        self._create_directories()

        for i in tqdm(range(img_sets)):
            for angle in self.angles:
                for j in range(self.min_stripe_num, self.max_stripe_num + 1):
                    img = self.create_rotated_stripes(j, angle)
                    img.save(os.path.join(self.dir_path, f"{angle}/img_{j}_{i}_{tag}.png"))

    def create_rotated_stripes(self, num_stripes, angle):
        # Create a new blank image
        img = Image.new("RGB", (self.c_size, self.c_size), color=background_colour)
        draw = ImageDraw.Draw(img)
        # Get random thickness
        stripe_thickness = np.random.randint(self.min_thickness, self.max_thickness, num_stripes)

        min_start_point = (self.c_size - self.size) // 2 * np.cos(angle * np.pi / 180)
        max_start_point = self.c_size - min_start_point - self.min_thickness - self.min_spacing
        starting_positions = np.random.randint(min_start_point, max_start_point, num_stripes)
        # Check if any stripes overlap
        attemps = 0
        while self._check_overlaps(starting_positions, stripe_thickness):
            starting_positions = np.random.randint(
                min_start_point, max_start_point, num_stripes
            )
            attemps += 1
            if attemps > max_attemps:
                raise Exception("Too many overlaps")

        # Draw vertical stripes
        for i in range(num_stripes):
            upper_left = (starting_positions[i], 0)
            lower_right = (
                starting_positions[i] + stripe_thickness[i],
                self.c_size,
            )
            draw.rectangle([upper_left, lower_right], fill="white")  # Stripe color

        # Rotate the image by the specified angle
        rotated_img = img.rotate(angle)
        # Crop the image to the original size
        rotated_img = rotated_img.crop(
            (
                (self.c_size - self.size) // 2,
                (self.c_size - self.size) // 2,
                (self.c_size + self.size) // 2,
                (self.c_size + self.size) // 2,
            )
        )

        return rotated_img

    def _create_directories(self):
        os.makedirs(self.dir_path, exist_ok=True)

        for angle in self.angles:
            os.makedirs(os.path.join(self.dir_path, f"{angle}"), exist_ok=True)

    def _check_overlaps(self, starting_positions, stripe_thickness):
        # Check if any stripes overlap
        for i in range(len(starting_positions)):
            for j in range(i + 1, len(starting_positions)):
                if (
                    starting_positions[i] < starting_positions[j] + stripe_thickness[j] + self.min_spacing
                    and starting_positions[i] + stripe_thickness[i] + self.min_spacing > starting_positions[j]
                ):
                    return True
        return False

In [74]:
# create images
stripes = CreateStripes(10, angles)
stripes.create_images()

100%|██████████| 50/50 [00:13<00:00,  3.73it/s]


In [37]:
np.linspace(10, 600, 4)[1:-1]

array([206.66666667, 403.33333333])